In [1]:
%%capture
from numpy import int16
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime
from threading import Timer
from pandas import read_excel
from siuba import _

from oactf import actif, patient, driver, SDATA

date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [2]:
""" def the_upload():
    year = ['2022-2023','2023-2024']
    schooling_dreams = read_excel(f"~/Downloads/schooling_dreams {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")
    schooling_dreams = schooling_dreams[schooling_dreams['schooling_year'].isin(year)]
    schooling_dreams = schooling_dreams[schooling_dreams.closed == False]
    payed_schooling_dreams = schooling_dreams[schooling_dreams.eskew_peye == "1"]
    return schooling_dreams, payed_schooling_dreams

t = Timer(3000.0,the_upload)
t.start()
schooling_dreams,payed_schooling_dreams = the_upload() """

year = ['2022-2023','2023-2024']
schooling_dreams = read_excel(f"~/Downloads/schooling_dreams {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")
schooling_dreams = schooling_dreams[schooling_dreams['schooling_year'].isin(year)]
schooling_dreams = schooling_dreams[schooling_dreams.closed == False]
payed_schooling_dreams = schooling_dreams[schooling_dreams.eskew_peye == "1"]

' def the_upload():\n    year = [\'2022-2023\',\'2023-2024\']\n    schooling_dreams = read_excel(f"~/Downloads/schooling_dreams {str(datetime.today().strftime(\'%Y-%m-%d\'))}.xlsx")\n    schooling_dreams = schooling_dreams[schooling_dreams[\'schooling_year\'].isin(year)]\n    schooling_dreams = schooling_dreams[schooling_dreams.closed == False]\n    payed_schooling_dreams = schooling_dreams[schooling_dreams.eskew_peye == "1"]\n    return schooling_dreams, payed_schooling_dreams\n\nt = Timer(3000.0,the_upload)\nt.start()\nschooling_dreams,payed_schooling_dreams = the_upload() '

In [3]:
driver.quit()

In [4]:
schooling_dreams.closed.unique()
schooling_dreams.eskew_peye.unique()
payed_schooling_dreams.eskew_peye.unique()

array([False])

array(['1', '---'], dtype=object)

array(['1'], dtype=object)

In [5]:
actif.id_patient.count()
patient.id_patient.count()
schooling_dreams.caseid.count()
payed_schooling_dreams.caseid.count()

7397

176593

533

464

In [6]:
actif.to_excel('actif.xlsx',index=False)

In [7]:
comp = actif[actif.complete_at_least=="yes"]
comp.id_patient.count()
comp.test_results.fillna("0,",inplace=True)
comp.test_results.value_counts()

7344

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_9534/319073030.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp.test_results.fillna("0,",inplace=True)


0,       6867
0,,2,     255
2,        217
0,,3,       2
1,          1
2,,3,       1
3,          1
Name: test_results, dtype: int64

In [8]:
ovc_data= comp
neg_data = comp[
    (comp.test_results!="0,,3,")&
    (comp.test_results!="0,,1,,2,")&
    (comp.test_results!="0,,2,,3,")&
    (comp.test_results!="3,")&
    (comp.test_results!="2,,3,")
]
ovc_data.id_patient.count()
neg_data.id_patient.count()
neg_data.test_results.value_counts()

7344

7340

0,       6867
0,,2,     255
2,        217
1,          1
Name: test_results, dtype: int64

In [9]:
positive_dreams= ovc_data[
    (ovc_data.test_results=="1,")|
    (ovc_data.test_results=="0,,1,")
]
positive_dreams.id_patient.count()

1

In [10]:
positive_dreams

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,ps_1519,ps_2024,secondary_1014,secondary_1519,secondary_2024,complete_1014,complete_1519,complete_2024,complete_at_least,isEnrolledQ2
561,175251.0,NaN,Ouest,Port-au-Prince,21,0,yes,yes,1,1,...,no,primary,no,no,secondary,no,no,no,yes,yes


In [11]:
ovc_data.ovc_age.unique()
ovc_data.parenting.unique()
ovc_data.parenting_detailed.unique()

array(['10-14', '15-17', '18-24', '25-29', 'not_valid_age'], dtype=object)

array(['no', 'yes'], dtype=object)

array(['no', 'yes', 'has_started'], dtype=object)

In [12]:
ovcS_dreams = ovc_data[
    (ovc_data.ovc_age=="10-14")|
    (ovc_data.ovc_age=="15-17")
]
ovcS_dreams['Gender'] = "Female"
ovcS_dreams.id_patient.count()

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_9534/39440959.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ovcS_dreams['Gender'] = "Female"


4661

In [13]:
ovcS_dreams = patient.merge(ovcS_dreams,on="id_patient",how="right")


In [14]:
ovcS_dreams[ovcS_dreams.dreams_code.isin(payed_schooling_dreams.dreams_code)].id_patient.count()
ovcS_dreams = ovcS_dreams[~ovcS_dreams.dreams_code.isin(payed_schooling_dreams.dreams_code)]
ovcS_dreams.id_patient.count()

331

4330

In [15]:
ovcS_dreams.parenting_detailed.unique()

array(['no', 'yes', 'has_started'], dtype=object)

In [16]:
datim_ovc_dreams = ovcS_dreams.pivot_table(index="commune",values="id_patient",columns=["Gender","ovc_age"],aggfunc='count',fill_value=0)
datim_ovc_dreams

Gender                         Female      
ovc_age                         10-14 15-17
commune                                    
Cap-Haïtien                       155   130
Carrefour                         122   104
Delmas                             35    40
Desdunes                           46    30
Dessalines                        212    73
Grande Saline                     117    78
Gressier                           96    60
Kenscoff                          427   198
La Chapelle                       111    90
Liancourt                          18    41
Limonade                           23    17
Milot                              13    17
Montrouis                          73    57
Petite Rivière de l'Artibonite     27    24
Plaine-du-Nord                     26    14
Port-au-Prince                    134    99
Pétionville                        71   101
Quartier-Morin                     25    10
Saint-Marc                        338   365
Tabarre                            57    24
Verrettes                         378   254

In [17]:
datim_ovc_dreams[('Female','<1')] = 0
datim_ovc_dreams[('Female','1-4')] = 0
datim_ovc_dreams[('Female','5-9')] = 0
datim_ovc_dreams[('Female','18+')] = 0
datim_ovc_dreams[('Male','<1')] = 0
datim_ovc_dreams[('Male','1-4')] = 0
datim_ovc_dreams[('Male','5-9')] = 0
datim_ovc_dreams[('Male','10-14')] = 0
datim_ovc_dreams[('Male','15-17')] = 0
datim_ovc_dreams[('Male','18+')] = 0

columns_orientation =[
    ('Female','<1'),
    ('Female','1-4'),
    ('Female','5-9'),
    ('Female','10-14'),
    ('Female','15-17'),   
    ('Female','18+'),  
    ('Male','<1'),
    ('Male','1-4'),
    ('Male','5-9'),
    ('Male','10-14'),
    ('Male','15-17'),   
    ('Male','18+'),   
]

datim_ovc_dreams = datim_ovc_dreams.reindex(columns= columns_orientation)
datim_ovc_dreams

Gender                         Female                         Male          \
ovc_age                            <1 1-4 5-9 10-14 15-17 18+   <1 1-4 5-9   
commune                                                                      
Cap-Haïtien                         0   0   0   155   130   0    0   0   0   
Carrefour                           0   0   0   122   104   0    0   0   0   
Delmas                              0   0   0    35    40   0    0   0   0   
Desdunes                            0   0   0    46    30   0    0   0   0   
Dessalines                          0   0   0   212    73   0    0   0   0   
Grande Saline                       0   0   0   117    78   0    0   0   0   
Gressier                            0   0   0    96    60   0    0   0   0   
Kenscoff                            0   0   0   427   198   0    0   0   0   
La Chapelle                         0   0   0   111    90   0    0   0   0   
Liancourt                           0   0   0    18    41   0    0   0   0   
Limonade                            0   0   0    23    17   0    0   0   0   
Milot                               0   0   0    13    17   0    0   0   0   
Montrouis                           0   0   0    73    57   0    0   0   0   
Petite Rivière de l'Artibonite      0   0   0    27    24   0    0   0   0   
Plaine-du-Nord                      0   0   0    26    14   0    0   0   0   
Port-au-Prince                      0   0   0   134    99   0    0   0   0   
Pétionville                         0   0   0    71   101   0    0   0   0   
Quartier-Morin                      0   0   0    25    10   0    0   0   0   
Saint-Marc                          0   0   0   338   365   0    0   0   0   
Tabarre                             0   0   0    57    24   0    0   0   0   
Verrettes                           0   0   0   378   254   0    0   0   0   

Gender                                          
ovc_age                        10-14 15-17 18+  
commune                                         
Cap-Haïtien                        0     0   0  
Carrefour                          0     0   0  
Delmas                             0     0   0  
Desdunes                           0     0   0  
Dessalines                         0     0   0  
Grande Saline                      0     0   0  
Gressier                           0     0   0  
Kenscoff                           0     0   0  
La Chapelle                        0     0   0  
Liancourt                          0     0   0  
Limonade                           0     0   0  
Milot                              0     0   0  
Montrouis                          0     0   0  
Petite Rivière de l'Artibonite     0     0   0  
Plaine-du-Nord                     0     0   0  
Port-au-Prince                     0     0   0  
Pétionville                        0     0   0  
Quartier-Morin                     0     0   0  
Saint-Marc                         0     0   0  
Tabarre                            0     0   0  
Verrettes                          0     0   0

In [18]:
#positive_dreams.to_excel('./ovc_results/positive_data/positive_dreams.xlsx',na_rep="",index=False)
#datim_ovc_dreams.to_excel('./ovc_results/datim_ovc_dreams.xlsx',na_rep="",index=True)

ovcS_dreams.to_excel(f"./ovc_results/valid_ovc_data.xlsx",na_rep="",index=False)
positive_dreams.to_excel(f"./ovc_results/positive_data/positive_dreams.xlsx",na_rep="",index=False)
datim_ovc_dreams.to_excel(f"./ovc_results/datim_ovc_dreams.xlsx",na_rep="",index=True)
""" ovcS_dreams.to_excel(f"./ovc_results/valid_ovc_data_{date_du_jour}.xlsx",na_rep="",index=False)
positive_dreams.to_excel(f"./ovc_results/positive_data/positive_dreams_{date_du_jour}.xlsx",na_rep="",index=False)
datim_ovc_dreams.to_excel(f"./ovc_results/datim_ovc_dreams_{date_du_jour}.xlsx",na_rep="",index=True)
 """


' ovcS_dreams.to_excel(f"./ovc_results/valid_ovc_data_{date_du_jour}.xlsx",na_rep="",index=False)\npositive_dreams.to_excel(f"./ovc_results/positive_data/positive_dreams_{date_du_jour}.xlsx",na_rep="",index=False)\ndatim_ovc_dreams.to_excel(f"./ovc_results/datim_ovc_dreams_{date_du_jour}.xlsx",na_rep="",index=True)\n '

# export positive ID

In [19]:
from siuba import _

In [20]:
export_positive = SDATA[_.id_patient.isin(positive_dreams.id_patient)]
export_positive.shape[0]

1

In [21]:
export_positive.to_excel(f"./ovc_results/positive_data/export_positive.xlsx",index=False)
#export_positive.to_excel(f"./ovc_results/positive_data/export_positive_{date_du_jour}.xlsx",index=False)